In [1]:
import sys
sys.path.append("../")

In [2]:
from utils.chroma_store import ChromaStore
from chromadb.utils import embedding_functions
from transformers import pipeline
import pandas as pd
import time
import requests
import json
import uuid
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('max_colwidth', 800)

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [3]:
### Parameters
COLLECTION_NAME_MAP = {"mind_data": "Mind", "nhs_data": "NHS"}
DEFAULT_EMBED_INSTRUCTION = "Represent the document for retrieval: "
DEFAULT_QUERY_INSTRUCTION = (
    "Represent the question for retrieving supporting documents: "
)

CHROMA_ENDPOINT = "localhost"
chroma_client = ChromaStore(chroma_server_hostname=CHROMA_ENDPOINT, chroma_server_port=8000)
SELDON_ENDPOINT = "http://localhost:9000/v2/models/transformer/infer"

MIND_CHUNK_SIZE = 780
MIND_CHUNK_OVERLAP = 50
NHS_CHUNK_SIZE = 2000
NHS_CHUNK_OVERLAP = 50

EMBED_MODEL = "hkunlp/instructor-base"
EMBED_FUNC = embedding_functions.InstructorEmbeddingFunction(model_name=EMBED_MODEL, instruction=DEFAULT_QUERY_INSTRUCTION)
N_CLOSEST_MATCHES = 3

QUESTIONS = [
"Can you tell me about depression?",
"What lifestyle changes are suggested for depression?",
"Are there different types of depressions?"
]

TEMPLATE = "Context: {context}\n\nQuestion: {question}\n\n"
MAX_OUTPUT_LENGTH = 300
TEMPERATURE = 0

Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
def use_local_base_model(messages):
    context = messages["context"]
    input_text = TEMPLATE.format(question=messages["prompt_query"], context=context)
    print(f"Prompt to LLM : {input_text}")
    llm = pipeline(model="google/flan-t5-base", task="text2text-generation")
    st = time.perf_counter()
    data = llm(input_text, max_length=MAX_OUTPUT_LENGTH, temperature=TEMPERATURE)
    time_taken = time.perf_counter()-st
    print(f"Time taken to generate response: {time_taken} sec")
    return time_taken, data[0]["generated_text"]

def query_vector_store(chroma_client, query_text, collection_name, n_results, embedding_function):
    result_dict = chroma_client.query_collection(
        collection_name=collection_name,
        query_texts=query_text,
        n_results=n_results,
        embedding_function=embedding_function,
    )
    documents = " ".join(result_dict["documents"][0])
    return documents


def _create_payload(messages):
    context = messages["context"]
    input_text = TEMPLATE.format(question=messages["prompt_query"], context=context)
    print(f"Prompt to LLM : {input_text}")
    return {
        "inputs": [
            {
                "name": "array_inputs",
                "shape": [-1],
                "datatype": "string",
                "data": str(input_text),
            },
            {
                "name": "max_length",
                "shape": [-1],
                "datatype": "INT32",
                "data": [MAX_OUTPUT_LENGTH],
                "parameters": {"content_type": "raw"},
            },
            {
                "name": "temperature",
                "shape": [-1],
                "datatype": "INT32",
                "data": [TEMPERATURE],
                "parameters": {"content_type": "raw"},
            },
        ]
    }


def _get_predictions(prediction_endpoint, payload):
    st = time.perf_counter()
    response = requests.post(
        url=prediction_endpoint,
        data=json.dumps(payload),
        headers={"Content-Type": "application/json"},
    )
    data = json.loads(json.loads(response.text)["outputs"][0]["data"][0])
    time_taken = time.perf_counter()-st
    print(f"Time taken to generate response: {time_taken} sec")
    return time_taken, data["generated_text"]


def query_llm(prediction_endpoint, messages):
    payload = _create_payload(messages)
    time_taken, summary_txt = _get_predictions(prediction_endpoint, payload)
    return time_taken, summary_txt


def run_experiment(questions, is_baseline = False):
    mind_responses, nhs_responses, mind_time, nhs_time = [], [], [], []
    for collection, source in COLLECTION_NAME_MAP.items():
        for prompt in questions:
            # Query vector store
            context = query_vector_store(
                chroma_client=chroma_client,
                query_text=prompt,
                collection_name=collection,
                n_results=N_CLOSEST_MATCHES,
                embedding_function=EMBED_FUNC,
            )
            
            # Create a dict of prompt and context
            message = {"prompt_query": prompt, "context": context}
            
            # Query LLM by passing query and context
            if is_baseline:
                time_taken, assistant_response = use_local_base_model(message)
            else:
                time_taken, assistant_response = query_llm(prediction_endpoint=SELDON_ENDPOINT, messages=message)
            
            print(f"Question: {prompt}")
            print(f"LLM Response: {assistant_response}")
            if source == "Mind":
                mind_responses.append(assistant_response)
                mind_time.append(round(time_taken, 2))
            else:
                nhs_responses.append(assistant_response)
                nhs_time.append(round(time_taken, 2))

            print("="*80)

    result_df = pd.DataFrame()
    result_df['questions'] = questions
    result_df['mind_responses'] = mind_responses
    result_df['nhs_responses'] = nhs_responses
    result_df['mind_time'] = mind_time
    result_df['nhs_time'] = nhs_time
    return result_df

# Experiments

> Note: This requires a local chroma store running in the background. To run a chroma store server, run the following steps

```bash
docker pull ghcr.io/chroma-core/chroma:0.4.3
docker run -it -p 8000:8000 ghcr.io/chroma-core/chroma:0.4.3
```

This will start chroma server on localhost at port 8000.

Run the embedding pipeline in a separate terminal to store the embeddings for the latest version of dataset.

```bash
python run.py --embed
```

> Note: Inside `data_embedding_pipeline.py`, comment out the `compute_embedding_drift` step as its not required.

### Flan-T5 Base Baseline

In [5]:
result_df = run_experiment(QUESTIONS, is_baseline=True)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated 

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 6.963960506996955 sec
Question: Can you tell me about depression?
LLM Response: Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood difficult experience. It may also feel hard to explain your thoughts and feelings to others.
Prompt to LLM : Context: to exercise, but research has shown that exercise can help reduce symptoms of depression. You may also find that it might help you to relax.  My diet has changed loads. I

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 2.630569346001721 sec
Question: Are there different types of depressions?
LLM Response: There are many different mental health problems. Some of them have similar symptoms, so you may experience the symptoms of more than one mental health problem, or be given several diagnoses at once. Or you might not have any particular diagnosis, but still be finding things very difficult.
Prompt to LLM : Context: Overview - Depression in adults. Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most peopl

...

Time taken to generate response: 3.7605805760103976 sec
Question: Can you tell me about depression?
LLM Response: Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Prompt to LLM : Context: Living with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depression could return. Stopping suddenly can also increase the risk of withdrawal side effects. Talk to a doctor or pharmacist if you have any questions or concerns about the medicine you're taking. The leaflet that comes with your medicine will have information about possible interactions with other medicines or supplements. Check with a doct

Token indices sequence length is longer than the specified maximum sequence length for this model (1226 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 2.3216000600077678 sec
Question: What lifestyle changes are suggested for depression?
LLM Response: Get more exercise, cutting down on alcohol, giving up smoking and eating healthily
Prompt to LLM : Context: are. how long depression lasts. the impact on your daily life. A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression. It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some other symptoms of depression are rare with grie

Token indices sequence length is longer than the specified maximum sequence length for this model (1326 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 4.814348858009907 sec
Question: Are there different types of depressions?
LLM Response: There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines.


In [6]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood difficult experience. It may also feel hard to explain your thoughts and feelings to others.","Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",6.96,3.76
1,What lifestyle changes are suggested for depression?,I gave up red meat and try to eat no sugar and drink hardly any alcohol. I exercise when I can and find meditation and yoga really helpful.,"Get more exercise, cutting down on alcohol, giving up smoking and eating healthily",1.95,2.32
2,Are there different types of depressions?,"There are many different mental health problems. Some of them have similar symptoms, so you may experience the symptoms of more than one mental health problem, or be given several diagnoses at once. Or you might not have any particular diagnosis, but still be finding things very difficult.","There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines.",2.63,4.81


## 8 vCPU

VM used : **Standard_D3_v12**

8 vCPU, 56 GB RAM, 112 GB Storage

### Flan-T5 Large

#### Prompt Template 1

Template: 

In [5]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated 

In [6]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things.,Depression is more than simply feeling unhappy or fed up for a few days.,3.76,4.74
1,What lifestyle changes are suggested for depression?,exercise,"getting more exercise , cutting down on alcohol , giving up smoking and eating healthily",1.35,4.59
2,Are there different types of depressions?,"There are many different mental health problems. Some of them have similar symptoms, so you may experience the symptoms of more than one mental health problem, or be given several diagnoses at once.","There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines.",4.45,10.06


### Flan-T5 XL

In [ ]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

In [ ]:
result_df

### Flan-T5 Large

In [5]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/dudeperf3ct/.cache/pypoetry/virtualenvs/mindgpt-N4bnLkub-py3.10/lib/python3.10/site-packag │
│ es/urllib3/connectionpool.py:714 in urlopen                                                      │
│                                                                                                  │
│    711 │   │   │   │   self._prepare_proxy(conn)                                                 │
│    712 │   │   │                                                                                 │
│    713 │   │   │   # Make the request on the httplib connection object.                          │
│ ❱  714 │   │   │   httplib_response = self._make_request(                                        │
│    715 │   │   │   │   conn,                                                                     │
│    716 │   │   │   │   method,                                                                   │
│    717 │   │   │   │   url,                                                                      │
│                                                                                                  │
│ /home/dudeperf3ct/.cache/pypoetry/virtualenvs/mindgpt-N4bnLkub-py3.10/lib/python3.10/site-packag │
│ es/urllib3/connectionpool.py:466 in _make_request                                                │
│                                                                                                  │
│    463 │   │   │   │   │   # Remove the TypeError from the exception chain in                    │
│    464 │   │   │   │   │   # Python 3 (including for exceptions like SystemExit).                │
│    465 │   │   │   │   │   # Otherwise it looks like a bug in the code.                          │
│ ❱  466 │   │   │   │   │   six.raise_from(e, None)                                               │
│    467 │   │   except (SocketTimeout, BaseSSLError, SocketError) as e:                           │
│    468 │   │   │   self._raise_timeout(err=e, url=url, timeout_value=read_timeout)               │
│    469 │   │   │   raise                                                                         │
│ <string>:3 in raise_from                                                                         │
│                                                                                                  │
│ /home/dudeperf3ct/.cache/pypoetry/virtualenvs/mindgpt-N4bnLkub-py3.10/lib/python3.10/site-packag │
│ es/urllib3/connectionpool.py:461 in _make_request                                                │
│                                                                                                  │
│    458 │   │   │   except TypeError:                                                             │
│    459 │   │   │   │   # Python 3                                                                │
│    460 │   │   │   │   try:                                                                      │
│ ❱  461 │   │   │   │   │   httplib_response = conn.getresponse()                                 │
│    462 │   │   │   │   except BaseException as e:                                                │
│    463 │   │   │   │   │   # Remove the TypeError from the exception chain in                    │
│    464 │   │   │   │   │   # Python 3 (including for exceptions like SystemExit).                │
│                                                                                                  │
│ /home/dudeperf3ct/.pyenv/versions/3.10.5/lib/python3.10/http/client.py:1374 in getresponse       │
│                                                                                                  │
│   1371 │   │                                                                                     │
│   1372 │   │   try:                                                                              │
│   1373 │   │   │   try:                                    

In [ ]:
result_df

### Flan-T5 XL (OOM)

In [ ]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

In [ ]:
result_df

### Flan-T5 Large

In [ ]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

In [ ]:
result_df

### Flan-T5 XL

In [ ]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

In [ ]:
result_df